## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Severo-Kurilsk,RU,50.6789,156.1250,52.45,82,66,12.33,broken clouds
1,1,Polunochnoye,RU,60.8697,60.4153,27.37,97,99,4.32,overcast clouds
2,2,Norman Wells,CA,65.2820,-126.8329,35.64,69,75,5.75,broken clouds
3,3,Tuktoyaktuk,CA,69.4541,-133.0374,33.80,86,75,13.80,light rain
4,4,Ponta Do Sol,PT,32.6667,-17.1000,70.43,70,27,3.36,scattered clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 76
What is the maximum temperature you would like for your trip? 84


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Garowe,SO,8.4054,48.4845,78.08,57,9,16.75,clear sky
11,11,Bambanglipuro,ID,-7.9500,110.2833,80.74,72,67,13.85,broken clouds
20,20,Sao Miguel Do Araguaia,BR,-13.2750,-50.1628,83.55,41,67,3.53,broken clouds
23,23,Kapaa,US,22.0752,-159.3190,81.19,72,19,4.00,light rain
24,24,Hithadhoo,MV,-0.6000,73.0833,82.20,74,100,17.56,overcast clouds
28,28,Atuona,PF,-9.8000,-139.0333,78.12,72,6,17.22,clear sky
30,30,Yulara,AU,-25.2406,130.9889,80.44,12,100,14.97,overcast clouds
35,35,Pringsewu,ID,-5.3582,104.9744,80.56,80,90,0.87,overcast clouds
39,39,Bengkulu,ID,-3.8004,102.2655,78.04,84,67,4.50,broken clouds
46,46,Port Blair,IN,11.6667,92.7500,77.04,94,40,2.30,mist


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
#index returned is empty, indicating no empty rows. 
preferred_cities_df[preferred_cities_df['Humidity'] == ""].index

Int64Index([], dtype='int64')

In [50]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#No empty rows!

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Garowe,SO,78.08,clear sky,8.4054,48.4845,
11,Bambanglipuro,ID,80.74,broken clouds,-7.9500,110.2833,
20,Sao Miguel Do Araguaia,BR,83.55,broken clouds,-13.2750,-50.1628,
23,Kapaa,US,81.19,light rain,22.0752,-159.3190,
24,Hithadhoo,MV,82.20,overcast clouds,-0.6000,73.0833,
28,Atuona,PF,78.12,clear sky,-9.8000,-139.0333,
30,Yulara,AU,80.44,overcast clouds,-25.2406,130.9889,
35,Pringsewu,ID,80.56,overcast clouds,-5.3582,104.9744,
39,Bengkulu,ID,78.04,broken clouds,-3.8004,102.2655,
46,Port Blair,IN,77.04,mist,11.6667,92.7500,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.head(40)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Garowe,SO,78.08,clear sky,8.4054,48.4845,Curubo Hotel
11,Bambanglipuro,ID,80.74,broken clouds,-7.9500,110.2833,Puri Brata
20,Sao Miguel Do Araguaia,BR,83.55,broken clouds,-13.2750,-50.1628,Hotel Executivo Palace
23,Kapaa,US,81.19,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
24,Hithadhoo,MV,82.20,overcast clouds,-0.6000,73.0833,Scoop Guest House
28,Atuona,PF,78.12,clear sky,-9.8000,-139.0333,Villa Enata
30,Yulara,AU,80.44,overcast clouds,-25.2406,130.9889,Desert Gardens Hotel - Ayers Rock Resort
35,Pringsewu,ID,80.56,overcast clouds,-5.3582,104.9744,Urban Style by Front One
39,Bengkulu,ID,78.04,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
46,Port Blair,IN,77.04,mist,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("", np.nan, inplace=True)


In [13]:
hotel_df.dropna(subset=["Hotel Name"], inplace=True)

In [14]:
hotel_df.head(40)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Garowe,SO,78.08,clear sky,8.4054,48.4845,Curubo Hotel
11,Bambanglipuro,ID,80.74,broken clouds,-7.9500,110.2833,Puri Brata
20,Sao Miguel Do Araguaia,BR,83.55,broken clouds,-13.2750,-50.1628,Hotel Executivo Palace
23,Kapaa,US,81.19,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
24,Hithadhoo,MV,82.20,overcast clouds,-0.6000,73.0833,Scoop Guest House
28,Atuona,PF,78.12,clear sky,-9.8000,-139.0333,Villa Enata
30,Yulara,AU,80.44,overcast clouds,-25.2406,130.9889,Desert Gardens Hotel - Ayers Rock Resort
35,Pringsewu,ID,80.56,overcast clouds,-5.3582,104.9744,Urban Style by Front One
39,Bengkulu,ID,78.04,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
46,Port Blair,IN,77.04,mist,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"


In [15]:
clean_hotel_df = hotel_df

In [16]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))